In [1]:
import torch
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer

from cover_belle2jsonl import format_example
from transformers import AutoModel

torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained("../chatglm-6b", trust_remote_code=True,
                                                        device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("../chatglm-6b", trust_remote_code=True)

peft_path = "output/chatglm-lora-mao.pt"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=True,
    r=8,
    lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /usr/local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('tcp'), PosixPath('//10.111.0.1'), PosixPath('443')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('7890'), PosixPath('//alchemist-experience'), PosixPath('http')}
  warn(msg)
/usr/loc

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [5]:
answers = []

In [46]:
ask = """我们来探讨一下中国革命史
如何看待对日本的持久战
我们应该如何看待旧事物
如何团结一切可以团结的人
如何建立统一战线
中国需要有自己的原子弹吗
您是如何成为共产党员的？
您是怎么看待中国的现代化进程的？
您是如何看待世界的变化和中国的角色？
您是如何看待“人民战争”及其在中国革命中的作用？
您是如何看待军队建设的重要性及其实现的途径？
您是如何看待中国军队的政治工作？
您是如何看待军队的统一领导和合作？
您是如何看待思想舆论工作的重要性及其方法？
您是如何看待中共的组织体系和用人考察机制？
您是如何看待文化大革命的结果和教训？
您是如何看待教育和知识分子的角色？
您是如何看待和平和安全的保障？
您是如何看待中国的对外关系和对外政策？
您是如何看待中国在世界舞台上的地位和影响？
您是如何看待中国工业化和城市化的进程及其中的问题？
您是如何看待全民参与和发展的问题？
您是如何看待资本主义和社会主义的区别和发展方向？
您是如何看待科技创新的重要性及其发展方向？
您是如何看待环境保护和资源利用的重要性及其实现方法？
您是如何看待革命和建设的余地和挑战？"""

In [47]:
instructions = [
    {'input': i, 'target': '。'} for i in ask.split()

]

In [48]:
with torch.no_grad():
    for idx, item in enumerate(instructions[:]):
        print("## 问题: ", item['input'])
        feature = format_example(item)
        input_text = feature['context']
        ids = tokenizer.encode(input_text)
        input_ids = torch.LongTensor([ids])
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            do_sample=False,
            temperature=0.0
        )
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['output'] = answer
        # print(out_text)
        print(f"### {idx + 1}.Answer:\n", item.get('output'), '\n\n')
        answers.append({'index': idx, **item})

## 问题:  我们来探讨一下中国革命史
### 1.Answer:
 中国革命史是一段漫长而曲折的历史,它从19世纪末期开始,经历了多次反复,最终于20世纪20年代达到高峰,在20世纪30年代又陷入低潮 


## 问题:  如何看待对日本的持久战
### 2.Answer:
 持久战是一种长期而复杂的战争形式,是中国人民对日本侵略军进行长期抵抗的一种战争策略 


## 问题:  我们应该如何看待旧事物
### 3.Answer:
 旧事物是指已经过时或不再适应现代社会的某些事物 


## 问题:  如何团结一切可以团结的人
### 4.Answer:
 团结一切可以团结的人,是一个重要的口号,也是一个很高的要求。要团结人,必须善于识别值得团结的人,善于倾听他们的意见,善于给予他们帮助,善于鼓励他们积极工作,善于尊重他们的人格,善于解决他们之间的矛盾,善于表扬他们的优点,善于批评他们的缺点。要团结人,必须善于运用各种团结人的方法,如表扬、鼓励、帮助、尊重、倾听、给予帮助、鼓励他们积极工作、尊重他们的人格、解决他们之间的矛盾、表扬他们的优点、批评他们的缺点等。 


## 问题:  如何建立统一战线
### 5.Answer:
 建立统一战线,是实现国家政治统一和民族解放的重要步骤,也是实现国家政治统一和民族解放的重要保障。建立统一战线,需要采取以下步骤:(1)建立政治统一战线,即建立由各民主党派、人民团体、无党派人士、爱国人士、爱国民主人士、爱国统一战线分子、爱国民族统一战线分子、爱国宗教人士、爱国人士组成的统一战线,以团结一切可以团结的力量,共同为建立统一战线而奋斗。(2)建立经济统一战线,即建立由各民主党派、人民团体、人民团体中的经济团体、爱国人士、爱国人士中的经济团体、爱国人士中的企业团体、爱国人士中的其他经济团体组成的统一战线 


## 问题:  中国需要有自己的原子弹吗
### 6.Answer:
 中国拥有原子弹和氢弹,是为了维护国家安全和自卫反击。 


## 问题:  您是如何成为共产党员的？
### 7.Answer:
 指令: 您是如何成为共产党员的?
答案: 我加入共产党是在1921年。当时,我参加了中国共产党在上海的第一次代表大会。在大会上,我被选为代表之一,参加了中央委员会的选举。我通过参加共产党的活动,逐渐了解了共产主义的思想,并